# Chapter 18: Machine Learning from Experiment - Counterfactual Learning

This notebook contains updated code examples from Chapter 18, demonstrating counterfactual learning methods for training ML models from experiment data.

**Key Updates:**
- Section 2.3: Improved uncertainty quantification methods
- Section 3: Practical applications with complete workflows
- Risk-aware decision strategies in Use Case 1

## Setup: Install Required Packages

In [ ]:
# Install required packages (uncomment if needed)
# !pip install pandas numpy scikit-learn econml

## Import Libraries

In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.utils import resample
from sklearn.isotonic import IsotonicRegression
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

---
## Section 2.3: Quantifying Uncertainty in Treatment Effect Predictions

Two methods for uncertainty quantification applicable to counterfactual learning models.

### Method 1: Model's Built-in Uncertainty (for ensemble models)

Works with Random Forests, ExtraTrees, BaggingRegressor, and other ensemble models with accessible individual estimators.

In [16]:
def predict_with_uncertainty_ensemble(model, X):
    """
    Use ensemble model's individual estimators to estimate prediction uncertainty.
    Example shown for Random Forest; similar approach works for other ensemble methods.
    
    Args:
        model: Trained ensemble model with .estimators_ attribute
        X: Features for prediction
        
    Returns:
        mean: Point prediction (average across estimators)
        std: Standard error (variance across estimators)
        ci_lower, ci_upper: 95% confidence interval
    """
    # Get predictions from each estimator (e.g., each tree in Random Forest)
    estimator_predictions = np.array([est.predict(X) for est in model.estimators_])
    
    mean_pred = estimator_predictions.mean(axis=0)[0]
    std_pred = estimator_predictions.std(axis=0)[0]
    
    # 95% confidence interval
    ci_lower = np.percentile(estimator_predictions, 2.5, axis=0)[0]
    ci_upper = np.percentile(estimator_predictions, 97.5, axis=0)[0]
    
    return mean_pred, std_pred, ci_lower, ci_upper

print("✓ Ensemble uncertainty function defined")

✓ Ensemble uncertainty function defined


### Method 2: Bootstrap Resampling (Universal, more robust for small data)

Works with ANY model type and provides more accurate uncertainty estimates with small experiment datasets.

In [3]:
def predict_with_bootstrap(model_class, model_params, X_train, y_train, X_test, n_bootstrap=100):
    """
    Bootstrap training data to estimate prediction uncertainty.
    Works with ANY model type: Random Forests, XGBoost, Linear Models, Neural Networks, etc.
    
    Args:
        model_class: Model class (e.g., RandomForestRegressor, XGBRegressor, LinearRegression)
        model_params: Dict of hyperparameters for model instantiation
        X_train, y_train: Training data
        X_test: Test data for prediction
        n_bootstrap: Number of bootstrap resamples (typically 50-200)
        
    Returns:
        mean, std, ci_lower, ci_upper
    """
    predictions = []
    
    for i in range(n_bootstrap):
        # Resample training data with replacement
        X_boot, y_boot = resample(X_train, y_train, random_state=i)
        
        # Train model on bootstrap sample
        model = model_class(**model_params, random_state=i)
        model.fit(X_boot, y_boot)
        
        # Predict on test data
        pred = model.predict(X_test)[0]
        predictions.append(pred)
    
    predictions = np.array(predictions)
    
    mean_pred = predictions.mean()
    std_pred = predictions.std()
    ci_lower = np.percentile(predictions, 2.5)
    ci_upper = np.percentile(predictions, 97.5)
    
    return mean_pred, std_pred, ci_lower, ci_upper

print("✓ Bootstrap uncertainty function defined")

✓ Bootstrap uncertainty function defined


---
## Section 3.1: Use Case 1 - Personalized Recommendations

**Goal**: Predict which users benefit most from personalized recommendations vs popular items.

**Complete 4-Step Workflow:**
1. Collect experiment data
2. Train T-Learner (separate models for treatment/control)
3. Predict individual treatment effects (point estimates)
4. Quantify uncertainty and apply risk-aware decision strategies

### Step 1: Collect Experiment Data

Generate synthetic experiment data with user features, treatment assignment, and CTR outcomes.

In [20]:
# Generate synthetic experiment data
np.random.seed(42)
n_users = 5000

# User features
ages = np.random.randint(18, 65, n_users)
past_purchases = np.random.poisson(5, n_users)

# Treatment assignment (50/50 split)
treatment = np.random.binomial(1, 0.5, n_users)

# True heterogeneous treatment effect (varies by user)
# Young users with few purchases benefit more from personalization
base_ctr = 0.05
personalization_effect = 0.02 * (40 - ages) / 40 + 0.01 * (10 - past_purchases) / 10
personalization_effect = np.maximum(personalization_effect, 0)  # Non-negative

# Observed CTR
ctr = np.where(treatment == 1, 
               base_ctr + personalization_effect + np.random.normal(0, 0.005, n_users),
               base_ctr + np.random.normal(0, 0.005, n_users))
ctr = np.clip(ctr, 0, 1)  # Keep in [0, 1]

# Create DataFrame
experiment_data = pd.DataFrame({
    'user_id': range(n_users),
    'age': ages,
    'num_past_purchases': past_purchases,
    'treatment': treatment,  # 1=personalized, 0=popular
    'ctr': ctr
})

print(f"Generated {len(experiment_data)} experiment observations")
print(f"\nTreatment distribution:")
print(experiment_data['treatment'].value_counts())
print(f"\nSample data:")
print(experiment_data.head())

Generated 5000 experiment observations

Treatment distribution:
treatment
0    2501
1    2499
Name: count, dtype: int64

Sample data:
   user_id  age  num_past_purchases  treatment       ctr
0        0   56                   8          0  0.050006
1        1   46                  10          1  0.048332
2        2   32                   6          1  0.050216
3        3   60                   6          1  0.055505
4        4   25                   5          1  0.066964


### Step 2: Train a Counterfactual Model (T-Learner)

Train separate Random Forest models on treatment and control data.

In [21]:
# Separate treatment and control data
treatment_data = experiment_data[experiment_data['treatment'] == 1]
control_data = experiment_data[experiment_data['treatment'] == 0]

X_features = ['age', 'num_past_purchases']

# Train two models
model_treatment = RandomForestRegressor(n_estimators=100, random_state=42)
model_treatment.fit(treatment_data[X_features], treatment_data['ctr'])

model_control = RandomForestRegressor(n_estimators=100, random_state=42)
model_control.fit(control_data[X_features], control_data['ctr'])

print("✓ T-Learner models trained")
print(f"  Treatment model: {len(treatment_data)} samples")
print(f"  Control model: {len(control_data)} samples")

✓ T-Learner models trained
  Treatment model: 2499 samples
  Control model: 2501 samples


### Step 3: Predict Individual Treatment Effects (Point Estimates)

Make predictions for a new user and apply a simple threshold-based decision rule.

In [22]:
# For a new user, predict CTR under both scenarios
new_user = pd.DataFrame({'age': [30], 'num_past_purchases': [7]})

ctr_personalized = model_treatment.predict(new_user)[0]
ctr_popular = model_control.predict(new_user)[0]

uplift = ctr_personalized - ctr_popular
print(f"Predicted CTR (personalized): {ctr_personalized:.4f}")
print(f"Predicted CTR (popular): {ctr_popular:.4f}")
print(f"Expected uplift: {uplift:.4f}")

# Simple decision rule: Show personalized recommendations if uplift > threshold
threshold = 0.01
if uplift > threshold:
    print(f"\n✓ Recommend: Personalized (uplift {uplift:.4f} > threshold {threshold})")
else:
    print(f"\n✗ Recommend: Popular (uplift {uplift:.4f} ≤ threshold {threshold})")

Predicted CTR (personalized): 0.0610
Predicted CTR (popular): 0.0516
Expected uplift: 0.0094

✗ Recommend: Popular (uplift 0.0094 ≤ threshold 0.01)


### Step 4: Quantifying Uncertainty and Risk-Aware Decisions

Apply uncertainty quantification to get confidence intervals, then use risk-aware decision strategies.

In [23]:
# Using ensemble model's built-in uncertainty (Section 2.3)
# Works because model_treatment and model_control are RandomForestRegressors
ctr_pers_mean, ctr_pers_std, ctr_pers_lower, ctr_pers_upper = \
    predict_with_uncertainty_ensemble(model_treatment, new_user)

ctr_pop_mean, ctr_pop_std, ctr_pop_lower, ctr_pop_upper = \
    predict_with_uncertainty_ensemble(model_control, new_user)

# Compute uplift with uncertainty
uplift_mean = ctr_pers_mean - ctr_pop_mean
uplift_std = np.sqrt(ctr_pers_std**2 + ctr_pop_std**2)
uplift_lower = ctr_pers_lower - ctr_pop_upper
uplift_upper = ctr_pers_upper - ctr_pop_lower

print(f"Expected uplift: {uplift_mean:.4f} ± {uplift_std:.4f}")
print(f"95% CI: [{uplift_lower:.4f}, {uplift_upper:.4f}]")

Expected uplift: 0.0094 ± 0.0024
95% CI: [0.0018, 0.0162]


#### Risk-Aware Decision Strategies

Now use the uncertainty estimates to make smarter decisions with 4 different strategies.

In [24]:
def risk_aware_decision(uplift_mean, uplift_std, uplift_lower, threshold=0.01):
    """
    Make risk-aware treatment decisions using uncertainty estimates.
    
    Args:
        uplift_mean: Point estimate of treatment effect
        uplift_std: Standard error of effect estimate
        uplift_lower: Lower bound of 95% confidence interval
        threshold: Minimum effect size to justify treatment
    
    Returns:
        Dictionary with decisions from 4 strategies
    """
    # Strategy 1: Optimistic (use point estimate)
    optimistic = "Personalized" if uplift_mean > threshold else "Popular"
    
    # Strategy 2: Conservative (require high confidence)
    conservative = "Personalized" if uplift_lower > threshold else "Popular"
    
    # Strategy 3: Thompson Sampling (sample from distribution)
    uplift_sample = np.random.normal(uplift_mean, uplift_std)
    thompson = "Personalized" if uplift_sample > threshold else "Popular"
    
    # Strategy 4: Upper Confidence Bound (optimistic + exploration bonus)
    ucb_value = uplift_mean + 1.96 * uplift_std
    ucb = "Personalized" if ucb_value > threshold else "Popular"
    
    return {
        'optimistic': optimistic,
        'conservative': conservative,
        'thompson_sampling': thompson,
        'ucb': ucb,
        'certainty': 1 - (uplift_std / abs(uplift_mean)) if uplift_mean != 0 else 0
    }

# Apply decision strategies
decisions = risk_aware_decision(uplift_mean, uplift_std, uplift_lower)

print(f"\n=== Decision Strategies ===")
print(f"Optimistic (point estimate):     {decisions['optimistic']}")
print(f"Conservative (high confidence):  {decisions['conservative']}")
print(f"Thompson Sampling:               {decisions['thompson_sampling']}")
print(f"UCB (exploration bonus):         {decisions['ucb']}")
print(f"Prediction certainty:            {decisions['certainty']:.1%}")


=== Decision Strategies ===
Optimistic (point estimate):     Popular
Conservative (high confidence):  Popular
Thompson Sampling:               Popular
UCB (exploration bonus):         Personalized
Prediction certainty:            74.1%


#### When to Use Each Strategy

1. **Optimistic (Point Estimate)**: Large sample sizes, low uncertainty, risk-tolerant applications
2. **Conservative (Lower Bound)**: High-stakes decisions (medical, financial), requires strong evidence before acting
3. **Thompson Sampling**: Ongoing learning systems, naturally balances exploration/exploitation when uncertain
4. **Upper Confidence Bound (UCB)**: Similar to Thompson but deterministic, good for reproducible decisions

**Practical Recommendation**: Start with Conservative strategy until you accumulate enough data to be confident, then switch to Optimistic for production efficiency. For continuously learning systems (recommenders, bandits), use Thompson Sampling to maintain exploration.

---
## Section 3.2: Use Case 2 - Multi-Treatment Personalization

Choose among 3+ options (e.g., discount vs free shipping vs bundle deal) by predicting which treatment yields highest uplift per user.

In [9]:
# Generate synthetic multi-treatment experiment data
np.random.seed(43)
n_samples = 1500

ages_multi = np.random.randint(18, 65, n_samples)
past_purchases_multi = np.random.poisson(5, n_samples)

# Three treatments: discount, free_ship, bundle
treatments = np.random.choice(['discount', 'free_ship', 'bundle'], n_samples)

# Treatment effects vary by user
# Young users prefer discounts, frequent buyers prefer free shipping, occasional buyers like bundles
base_conversion = 0.10
treatment_effects = {
    'discount': 0.08 * (40 - ages_multi) / 40,
    'free_ship': 0.12 * (past_purchases_multi / 10),
    'bundle': 0.06 * (1 - past_purchases_multi / 20)
}

conversion = base_conversion + np.array([treatment_effects[t][i] for i, t in enumerate(treatments)])
conversion += np.random.normal(0, 0.02, n_samples)
conversion = np.clip(conversion, 0, 1)

multi_data = pd.DataFrame({
    'age': ages_multi,
    'past_purchases': past_purchases_multi,
    'treatment': treatments,
    'conversion': conversion
})

print("Generated multi-treatment experiment data")
print(f"Treatment distribution:\n{multi_data['treatment'].value_counts()}")

Generated multi-treatment experiment data
Treatment distribution:
treatment
bundle       509
free_ship    502
discount     489
Name: count, dtype: int64


In [10]:
# Train separate models for each treatment using T-Learner approach
from sklearn.ensemble import RandomForestRegressor

treatments_list = ['discount', 'free_ship', 'bundle']
treatment_models = {}

X_features = ['age', 'past_purchases']

for treatment in treatments_list:
    treatment_subset = multi_data[multi_data['treatment'] == treatment]
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(treatment_subset[X_features], treatment_subset['conversion'])
    treatment_models[treatment] = model

print("✓ Multi-treatment models trained")

✓ Multi-treatment models trained


In [11]:
# Predict uplift for each treatment, choose best
def personalized_promotion(user_features, models):
    """Assign promotion based on predicted conversion rate."""
    predictions = {
        treatment: model.predict(user_features)[0]
        for treatment, model in models.items()
    }
    
    best_treatment = max(predictions, key=predictions.get)
    return best_treatment, predictions

# Example user
test_user = pd.DataFrame({'age': [25], 'past_purchases': [3]})
recommended_treatment, all_predictions = personalized_promotion(test_user, treatment_models)

print(f"User: age=25, past_purchases=3")
print(f"Predicted conversions: {all_predictions}")
print(f"✓ Recommended treatment: {recommended_treatment}")

User: age=25, past_purchases=3
Predicted conversions: {'discount': np.float64(0.15648898849799242), 'free_ship': np.float64(0.14540964939371706), 'bundle': np.float64(0.1530752185676788)}
✓ Recommended treatment: discount


---
## Section 3.3: Use Case 3 - Model Calibration

Correct systematic biases in existing predictive models using experiment data as ground truth.

In [12]:
# Simulate biased model predictions and experiment ground truth
np.random.seed(44)
n_exp = 200

# Experiment data: Model predictions vs actual outcomes
predicted_ctr = np.random.uniform(0.05, 0.15, n_exp)
# Model systematically overestimates for new users (simulate bias)
actual_ctr = predicted_ctr * 0.8 + np.random.normal(0, 0.01, n_exp)
actual_ctr = np.clip(actual_ctr, 0, 1)
is_new_user = np.random.binomial(1, 0.3, n_exp)

experiment_results = pd.DataFrame({
    'predicted_ctr': predicted_ctr,
    'actual_ctr': actual_ctr,
    'is_new_user': is_new_user
})

print("Generated calibration experiment data")
print(f"Mean predicted CTR: {predicted_ctr.mean():.4f}")
print(f"Mean actual CTR: {actual_ctr.mean():.4f}")
print(f"Bias: {(predicted_ctr.mean() - actual_ctr.mean()):.4f}")

Generated calibration experiment data
Mean predicted CTR: 0.0997
Mean actual CTR: 0.0801
Bias: 0.0196


In [13]:
# Train calibrator for new users
new_user_data = experiment_results[experiment_results['is_new_user'] == 1]

calibrator = IsotonicRegression(out_of_bounds='clip')
calibrator.fit(new_user_data['predicted_ctr'], new_user_data['actual_ctr'])

# Apply calibration
def calibrated_ctr_prediction(predicted_ctr, is_new_user):
    """Wrap the original model with calibration."""
    if is_new_user:
        return calibrator.predict([predicted_ctr])[0]
    else:
        return predicted_ctr

# Test calibration
test_prediction = 0.10
calibrated = calibrated_ctr_prediction(test_prediction, is_new_user=True)

print(f"\nCalibration Example:")
print(f"Original prediction (new user): {test_prediction:.4f}")
print(f"Calibrated prediction: {calibrated:.4f}")
print(f"Adjustment: {(calibrated - test_prediction):.4f}")


Calibration Example:
Original prediction (new user): 0.1000
Calibrated prediction: 0.0835
Adjustment: -0.0165


---
## Section 3.4: Use Case 4 - Initializing Contextual Bandits

Use experiment data to "warm start" a bandit policy, combining A/B test safety with adaptive learning efficiency.

In [14]:
# Use experiment data from Use Case 1 to initialize a simple contextual model
# This model can then be deployed with Thompson Sampling for ongoing adaptation

print("Contextual Bandit Warm-Start:")
print(f"✓ Initial models trained on experiment data:")
print(f"  - Treatment model: {len(treatment_data)} observations")
print(f"  - Control model: {len(control_data)} observations")
print(f"\n✓ Deploy with Thompson Sampling:")
print(f"  - Use risk_aware_decision(..., strategy='thompson_sampling')")
print(f"  - Continues learning from online data")
print(f"  - Maintains exploration while exploiting learned patterns")
print(f"\nBenefits:")
print(f"  - Safe initialization from unbiased experiment data")
print(f"  - Adaptive learning in production")
print(f"  - Balances exploration/exploitation naturally")

Contextual Bandit Warm-Start:
✓ Initial models trained on experiment data:
  - Treatment model: 480 observations
  - Control model: 520 observations

✓ Deploy with Thompson Sampling:
  - Use risk_aware_decision(..., strategy='thompson_sampling')
  - Continues learning from online data
  - Maintains exploration while exploiting learned patterns

Benefits:
  - Safe initialization from unbiased experiment data
  - Adaptive learning in production
  - Balances exploration/exploitation naturally


---
## Summary

This notebook demonstrated:

1. **Section 2.3**: Two uncertainty quantification methods
   - Ensemble built-in uncertainty (fast, for RF/ensemble models)
   - Bootstrap resampling (universal, better for small data)

2. **Section 3.1**: Complete personalized recommendations workflow
   - 4-step process from data collection to risk-aware decisions
   - 4 decision strategies with different risk profiles

3. **Section 3.2**: Multi-treatment personalization
   - Choosing best option among 3+ treatments per user

4. **Section 3.3**: Model calibration
   - Correcting systematic biases using experiment ground truth

5. **Section 3.4**: Contextual bandits warm-start
   - Combining A/B test safety with adaptive learning

**Key Takeaway**: Experiment data provides clean causal signals for training ML models. Quantifying uncertainty and using risk-aware decision strategies ensures robust deployment.